<a href="https://colab.research.google.com/github/skyloma/img2/blob/master/ColabAndTpuTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 授权访问google cloud partform

In [0]:
from google.colab import auth
auth.authenticate_user()


bucket_name="gs://staging.lomasky8090.appspot.com"

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
project_id = 'lomasky8090'
!gcloud config set project {project_id}

Updated property [core/project].


# github上加载models

In [0]:
!git clone https://github.com/skyloma/models.git  
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython 
!cd models/research; protoc object_detection/protos/*.proto --python_out=.

Cloning into 'models'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 2777 (delta 4), reused 8 (delta 3), pack-reused 2767
Receiving objects: 100% (2777/2777), 281.14 MiB | 42.27 MiB/s, done.
Resolving deltas: 100% (511/511), done.
Checking out files: 100% (2641/2641), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.16-2~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile


# 切换到目录research下进行训练

In [0]:
!pip install tensorboardcolab

In [0]:

from tensorboardcolab import TensorBoardColab
tbc=TensorBoardColab(graph_path='gs://staging.lomasky8090.appspot.com/ssd224')

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://629f2662.ngrok.io


In [0]:

!cd models/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/model_tpu_main.py \
--pipeline_config_path=ssd128.config \
--model_dir=gs://staging.lomasky8090.appspot.com/ssd128 \
--num_train_steps=10000 \
--num_eval_steps=8000

In [0]:
! ls
!tensorflow -version

adc.json  models  outdata  sample_data


# 导出训练生成的文件

In [0]:
!mkdir outdata
bucket_name="gs://staging.lomasky8090.appspot.com" 
!gsutil cp  -r {bucket_name}/tpu192  outdata/
  
# !gsutil cp  gs://{bucket_name}/data/pk_rmb.pbtxt A/
  
# !gsutil cp  gs://{bucket_name}/A/tflite_graph.pbtxt A/  

# 导出能转换成tflte的pb

In [0]:
!mkdir outmodel
!cd models/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=object_detection/tpu192.config \
--trained_checkpoint_prefix=/content/outdata/tpu192/model.ckpt-50000 \
--output_directory=/content/outmodel/tpu92/ \
--add_postprocessing_op=true

# 转换量化的tflite

In [0]:
!tflite_convert \
  --graph_def_file=/content/outmodel/tpu192/tflite_graph.pb \
  --output_format=TFLITE \
  --output_file=/content/outmodel/tpu192/tflite_graph.tflite \
  --input_arrays=normalized_input_image_tensor \
  --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
  --input_shapes=1,192,192,3 \
  --inference_type=QUANTIZED_UINT8 \
  --mean_value=128 \
  --std_dev_values=128 \
  --default_ranges_min=0 \
  --default_ranges_max=6 \
  --change_concat_input_ranges=false \
  --allow_custom_ops
 

# 下载模型到mytf4930工程 解压copy到gs://mytf_storage/data/

In [0]:
# cd /tmp
!curl -O http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03.tar.gz
!tar xzf ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03.tar.gz
bucket_name="gs://lomasky8090.appspot.com"
!gsutil cp -r ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03  {bucket_name}/data/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 44.4M  100 44.4M    0     0   107M      0 --:--:-- --:--:-- --:--:--  107M
Copying file://ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/model.ckpt.meta [Content-Type=application/octet-stream]...
Copying file://ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb [Content-Type=application/octet-stream]...
Copying file://ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/pipeline.config [Content-Type=application/octet-stream]...
Copying file://ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/checkpoint [Content-Type=application/octet-stream]...
\ [4 files][ 20.7 MiB/ 20.7 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the

In [0]:
!ls 

In [0]:
!git clone https://github.com/skyloma/img2.git  

In [0]:
!cd models/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/xml_to_csv.py   --image_dir=/content/img2/

In [0]:
%cd /content  
!ls


In [0]:
!mkdir data 

In [0]:
!cd models/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim;python object_detection/generate_tfrecord.py \
    --csv_input=/content/img2/train_labels.csv \
    --image_dir=/content/img2/train \
    --output_path=/content/data/train.record  
!cd models/research;  export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/generate_tfrecord.py \
    --csv_input=/content/img2/test_labels.csv \
    --image_dir=/content/img2/test \
    --output_path=/content/data/test.record  

In [0]:
%cd /content  
!ls

In [0]:
!gsutil cp -r /content/data {bucket_name}
!gsutil cp -r /content/models/research/object_detection/label.pbtxt {bucket_name}/data/ 


In [0]:
# !rm -r data
# !rm -r img2